In [0]:
from pyspark.sql import SparkSession,functions as func
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType,LongType,DoubleType

In [0]:
spark=SparkSession.builder.appName("Regression Models").getOrCreate()

In [0]:
links=spark.read.option("inferSchema","true").option("header","true").csv("dbfs:/FileStore/tables/links.csv")
links.show(5)

+-------+------+------+
movieId|imdbId|tmdbId|
+-------+------+------+
 1|114709| 862|
 2|113497| 8844|
 3|113228| 15602|
 4|114885| 31357|
 5|113041| 11862|
+-------+------+------+
only showing top 5 rows

In [0]:
links.printSchema()

root
-- movieId: integer (nullable = true)
-- imdbId: integer (nullable = true)
-- tmdbId: integer (nullable = true)

In [0]:
ratings=spark.read.option("inferSchema","true").option("header","true").csv("dbfs:/FileStore/tables/ratings.csv")
ratings.show(5)

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 1| 296| 5.0|1147880044|
 1| 306| 3.5|1147868817|
 1| 307| 5.0|1147868828|
 1| 665| 5.0|1147878820|
 1| 899| 3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
ratings.printSchema()

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: integer (nullable = true)

In [0]:
links.createOrReplaceTempView("link")
ratings.createOrReplaceTempView("rating")

In [0]:
join_df=spark.sql("select l.*,r.userId,r.rating from link l inner join rating r on l.movieId=r.movieId")
join_df.show(10)

+-------+------+------+------+------+
movieId|imdbId|tmdbId|userId|rating|
+-------+------+------+------+------+
 296|110912| 680| 1| 5.0|
 306|111495| 110| 1| 3.5|
 307|108394| 108| 1| 5.0|
 665|114787| 11902| 1| 5.0|
 899| 45152| 872| 1| 3.5|
 1088| 92890| 88| 1| 4.0|
 1175|101700| 892| 1| 3.5|
 1217| 89881| 11645| 1| 3.5|
 1237| 50976| 490| 1| 5.0|
 1250| 50212| 826| 1| 4.0|
+-------+------+------+------+------+
only showing top 10 rows

In [0]:
join_df.createOrReplaceTempView("temp_table")

In [0]:
genome_score=spark.read.option("inferSchema","true").option("header","true").csv("dbfs:/FileStore/tables/genome_score.csv")
genome_score.show(10)

+-------+-----+---------+
movieId|tagId|relevance|
+-------+-----+---------+
 1| 1| 0.02875|
 1| 2| 0.02375|
 1| 3| 0.0625|
 1| 4| 0.07575|
 1| 5| 0.14075|
 1| 6| 0.14675|
 1| 7| 0.0635|
 1| 8| 0.20375|
 1| 9| 0.202|
 1| 10| 0.03075|
+-------+-----+---------+
only showing top 10 rows

In [0]:
genome_score.createOrReplaceTempView("score")

In [0]:
final_df=spark.sql("select t.*,s.tagId,s.relevance from temp_table t inner join score s on t.movieId=s.movieId")
final_df.show(10)

+-------+------+------+------+------+-----+---------+
movieId|imdbId|tmdbId|userId|rating|tagId|relevance|
+-------+------+------+------+------+-----+---------+
 1|114709| 862| 2| 3.5| 1128| 0.022|
 1|114709| 862| 2| 3.5| 1127| 0.08475|
 1|114709| 862| 2| 3.5| 1126| 0.02975|
 1|114709| 862| 2| 3.5| 1125| 0.039|
 1|114709| 862| 2| 3.5| 1124| 0.05775|
 1|114709| 862| 2| 3.5| 1123| 0.14125|
 1|114709| 862| 2| 3.5| 1122| 0.035|
 1|114709| 862| 2| 3.5| 1121| 0.0305|
 1|114709| 862| 2| 3.5| 1120| 0.01425|
 1|114709| 862| 2| 3.5| 1119| 0.0405|
+-------+------+------+------+------+-----+---------+
only showing top 10 rows

In [0]:
final_df.count()

Out[20]: 4487481

In [0]:
final_df.printSchema()

root
-- movieId: integer (nullable = true)
-- imdbId: integer (nullable = true)
-- tmdbId: integer (nullable = true)
-- userId: integer (nullable = true)
-- rating: double (nullable = true)
-- tagId: integer (nullable = true)
-- relevance: double (nullable = true)

In [0]:
from pyspark.ml.regression import *
from pyspark.ml.feature import *
from pyspark.ml.linalg import *
from pyspark.ml.evaluation import *

In [0]:
featureAssembler=VectorAssembler(inputCols=["movieId","imdbId","tmdbId","userId","tagId","relevance"],outputCol="Independent Variables")
output=featureAssembler.transform(final_df)

In [0]:
output.show(5)

+-------+------+------+------+------+-----+---------+---------------------+
movieId|imdbId|tmdbId|userId|rating|tagId|relevance|Independent Variables|
+-------+------+------+------+------+-----+---------+---------------------+
 1|114709| 862| 2| 3.5| 1128| 0.022| [1.0,114709.0,862...|
 1|114709| 862| 2| 3.5| 1127| 0.08475| [1.0,114709.0,862...|
 1|114709| 862| 2| 3.5| 1126| 0.02975| [1.0,114709.0,862...|
 1|114709| 862| 2| 3.5| 1125| 0.039| [1.0,114709.0,862...|
 1|114709| 862| 2| 3.5| 1124| 0.05775| [1.0,114709.0,862...|
+-------+------+------+------+------+-----+---------+---------------------+
only showing top 5 rows

In [0]:
dataset=output.select("Independent Variables","rating")
dataset.show(5)

+---------------------+------+
Independent Variables|rating|
+---------------------+------+
 [1.0,114709.0,862...| 3.5|
 [1.0,114709.0,862...| 3.5|
 [1.0,114709.0,862...| 3.5|
 [1.0,114709.0,862...| 3.5|
 [1.0,114709.0,862...| 3.5|
+---------------------+------+
only showing top 5 rows

In [0]:
train_data,test_data=dataset.randomSplit([0.7,0.3])

In [0]:
# Applying Linear Regression Model.

In [0]:
lr=LinearRegression(featuresCol="Independent Variables",labelCol="rating",maxIter=10,regParam=0.5)
regressor=lr.fit(train_data)

In [0]:
print(regressor.coefficients)
print(regressor.intercept)

[-0.10801545865334342,0.00016597509544617053,-1.5271388892037032e-05,-2.369595328209368e-06,-3.252061816763325e-08,0.022889879202446457]
-15.061307753374185

In [0]:
pred=regressor.evaluate(test_data)
pred.predictions.show(5)

+---------------------+------+------------------+
Independent Variables|rating| prediction|
+---------------------+------+------------------+
 [1.0,114709.0,862...| 3.5| 3.857003386598107|
 [1.0,114709.0,862...| 3.5| 3.861008887814206|
 [1.0,114709.0,862...| 3.5| 3.860962945452714|
 [1.0,114709.0,862...| 3.5|3.8567224652479855|
 [1.0,114709.0,862...| 3.5|3.8635950213960495|
+---------------------+------+------------------+
only showing top 5 rows

In [0]:
evaluator=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
pred_result=regressor.transform(test_data)
rmse=evaluator.evaluate(pred_result)
print(rmse)

0.9434757242864568

In [0]:
#Applying Decision Tree Regressor.

In [0]:
dtr=DecisionTreeRegressor(featuresCol="Independent Variables",labelCol="rating")
model=dtr.fit(train_data)

In [0]:
print("Feature Importances:",model.featureImportances.toArray)

Feature Importances: <bound method SparseVector.toArray of SparseVector(6, {0: 0.9058, 3: 0.0941, 4: 0.0})>

In [0]:
predicts=model.transform(test_data)
predicts.show(5)

+---------------------+------+-----------------+
Independent Variables|rating| prediction|
+---------------------+------+-----------------+
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
+---------------------+------+-----------------+
only showing top 5 rows

In [0]:
new_evaluator=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
rmse_value=new_evaluator.evaluate(predicts)
print(rmse_value)

0.935723424755869

In [0]:
#Applying Random Forest Regressor.

In [0]:
rfr=RandomForestRegressor(featuresCol="Independent Variables",labelCol="rating")
new_model=rfr.fit(train_data)

In [0]:
print("Feature Importances :",new_model.featureImportances.toArray)

Feature Importances : <bound method SparseVector.toArray of SparseVector(6, {0: 0.4968, 1: 0.1855, 2: 0.2884, 3: 0.0267, 4: 0.0001, 5: 0.0025})>

In [0]:
new_predicts=new_model.transform(test_data)
new_predicts.show(5)

+---------------------+------+------------------+
Independent Variables|rating| prediction|
+---------------------+------+------------------+
 [1.0,114709.0,862...| 3.5|3.9224820025680835|
 [1.0,114709.0,862...| 3.5| 3.925927090909218|
 [1.0,114709.0,862...| 3.5| 3.925034923817216|
 [1.0,114709.0,862...| 3.5|3.9218259560427584|
 [1.0,114709.0,862...| 3.5|3.9266741896042547|
+---------------------+------+------------------+
only showing top 5 rows

In [0]:
evaluator2=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
calculated_rmse=evaluator2.evaluate(new_predicts)
print(calculated_rmse)

0.9387521706989937

In [0]:
# Applying Gradient-boosted tree regression.

In [0]:
gbt=GBTRegressor(featuresCol="Independent Variables",labelCol="rating",maxIter=10)
gbt_model=gbt.fit(train_data)

In [0]:
gbt_predicts=gbt_model.transform(test_data)
gbt_predicts.show(5)

+---------------------+------+------------------+
Independent Variables|rating| prediction|
+---------------------+------+------------------+
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
 [1.0,114709.0,862...| 3.5|3.9141464051393604|
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
+---------------------+------+------------------+
only showing top 5 rows

In [0]:
gbt_evaluator=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
gbt_rmse=gbt_evaluator.evaluate(gbt_predicts)
print(gbt_rmse)

0.9321227519475505

In [0]:
#Introducing MLFlow with Databricks.
# pip install mlflow

In [0]:
import mlflow

In [0]:
with mlflow.start_run():
  lr1=LinearRegression(featuresCol="Independent Variables",labelCol="rating",maxIter=5,regParam=0.1)
  regressor1=lr1.fit(train_data)
  print(regressor1.coefficients)
  print(regressor1.intercept)
  pred1=regressor1.evaluate(test_data)
  pred1.predictions.show(5)
  evaluator1=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
  pred_result1=regressor1.transform(test_data)
  rmse1=evaluator1.evaluate(pred_result1)
  print(rmse1)

[-0.0818688695978008,0.0002614291341638032,-1.4080008813830507e-05,-3.2558461439554663e-06,-1.3208031950784742e-06,0.006948767196186226]
-26.002386816277614
/databricks/spark/python/pyspark/sql/context.py:120: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)
+---------------------+------+------------------+
Independent Variables|rating| prediction|
+---------------------+------+------------------+
 [1.0,114709.0,862...| 3.5| 3.892073841884173|
 [1.0,114709.0,862...| 3.5| 3.89328063052114|
 [1.0,114709.0,862...| 3.5|3.8932601289707733|
 [1.0,114709.0,862...| 3.5| 3.891967586634898|
 [1.0,114709.0,862...| 3.5| 3.894048670772772|
+---------------------+------+------------------+
only showing top 5 rows

0.9418258830180407

In [0]:
with mlflow.start_run():
  dtr1=DecisionTreeRegressor(featuresCol="Independent Variables",labelCol="rating")
  model1=dtr1.fit(train_data)
  print("Feature Importances:",model1.featureImportances.toArray)
  predicts1=model1.transform(test_data)
  predicts1.show(5)
  new_evaluator1=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
  rmse_value1=new_evaluator1.evaluate(predicts1)
  print(rmse_value1)

Feature Importances: <bound method SparseVector.toArray of SparseVector(6, {0: 0.9058, 3: 0.0941, 4: 0.0})>
+---------------------+------+-----------------+
Independent Variables|rating| prediction|
+---------------------+------+-----------------+
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
 [1.0,114709.0,862...| 3.5|3.986653570603189|
+---------------------+------+-----------------+
only showing top 5 rows

0.935723424755869

In [0]:
with mlflow.start_run():
  rfr1=RandomForestRegressor(featuresCol="Independent Variables",labelCol="rating")
  new_model1=rfr1.fit(train_data)
  print("Feature Importances :",new_model1.featureImportances.toArray)
  new_predicts1=new_model1.transform(test_data)
  new_predicts1.show(5)
  new_evaluator2=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
  calculated_rmse1=new_evaluator2.evaluate(new_predicts1)
  print(calculated_rmse1)

Feature Importances : <bound method SparseVector.toArray of SparseVector(6, {0: 0.4968, 1: 0.1855, 2: 0.2884, 3: 0.0267, 4: 0.0001, 5: 0.0025})>
+---------------------+------+------------------+
Independent Variables|rating| prediction|
+---------------------+------+------------------+
 [1.0,114709.0,862...| 3.5|3.9224820025680835|
 [1.0,114709.0,862...| 3.5| 3.925927090909218|
 [1.0,114709.0,862...| 3.5| 3.925034923817216|
 [1.0,114709.0,862...| 3.5|3.9218259560427584|
 [1.0,114709.0,862...| 3.5|3.9266741896042547|
+---------------------+------+------------------+
only showing top 5 rows

0.9387521706989937

In [0]:
with mlflow.start_run():
  gbt1=GBTRegressor(featuresCol="Independent Variables",labelCol="rating",maxIter=10)
  gbt_model1=gbt1.fit(train_data)
  gbt_predicts1=gbt_model1.transform(test_data)
  gbt_predicts1.show(5)
  gbt_evaluator1=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
  gbt_rmse1=gbt_evaluator1.evaluate(gbt_predicts1)
  print(gbt_rmse1)

+---------------------+------+------------------+
Independent Variables|rating| prediction|
+---------------------+------+------------------+
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
 [1.0,114709.0,862...| 3.5|3.9141464051393604|
 [1.0,114709.0,862...| 3.5|3.9164602292679564|
+---------------------+------+------------------+
only showing top 5 rows

0.9321227519475505